In [ ]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
from sqlalchemy import create_engine, text

In [ ]:
pip install psycopg2-binary

# GEOSPATIAL ANALYSIS SECTION

In [ ]:
trips_cleaned = pd.read_csv('../data/tripscleaned.csv') # cleaned scooters trips data from 3/21/23 William
print(trips_cleaned.shape)
trips_cleaned.head(3)

In [ ]:
trips_cleaned_starts = trips_cleaned[['companyname', 'triprecordnum', 'sumdid', 'tripduration', 'tripdistance', 'startdate', 'starttime', 'startlatitude', 'startlongitude']]
trips_cleaned_starts.head()

In [ ]:
trips_cleaned_ends = trips_cleaned[['companyname', 'triprecordnum', 'sumdid', 'tripduration', 'tripdistance', 'enddate', 'endtime', 'endlatitude', 'endlongitude']]
trips_cleaned_ends.head()

In [ ]:
trips_cleaned_starts['geometry'] = trips_cleaned_starts.apply(lambda x: Point((float(x.startlongitude), 
                                                         float(x.startlatitude))), 
                                        axis=1)
trips_cleaned_starts.head(3)

In [ ]:
trips_cleaned_starts = gpd.GeoDataFrame(trips_cleaned_starts, geometry=gpd.points_from_xy(trips_cleaned_starts.startlongitude, trips_cleaned_starts.startlatitude))
trips_cleaned_starts.head()

In [ ]:
trips_cleaned_starts = trips_cleaned_starts.set_crs(4326)
print(trips_cleaned_starts.crs)

In [ ]:
trips_cleaned_starts.geometry.to_crs('EPSG:4326')
print(trips_cleaned_starts.crs)

In [ ]:
trips_cleaned_ends['geometry'] = trips_cleaned_ends.apply(lambda x: Point((float(x.endlongitude), 
                                                         float(x.endlatitude))), 
                                        axis=1)
trips_cleaned_ends.head(3)

In [ ]:
trips_cleaned_ends = gpd.GeoDataFrame(trips_cleaned_ends, geometry=gpd.points_from_xy(trips_cleaned_ends.endlongitude, trips_cleaned_ends.endlatitude))
trips_cleaned_ends.head()

In [ ]:
trips_cleaned_ends = trips_cleaned_ends.set_crs(4326)
print(trips_cleaned_ends.crs)

In [ ]:
trips_cleaned_ends.geometry.to_crs('EPSG:4326')
print(trips_cleaned_ends.crs)

In [ ]:
zipcodes = gpd.read_file('../data/zipcodes.geojson') # from earlier geospatial exercise
print(zipcodes.crs)
zipcodes.head( )

In [ ]:
zipcodes.plot();

In [ ]:
leg_kwds = {'title': 'Zipcodes', 'loc': 'upper left', 
            'bbox_to_anchor': (1, 1.03), 'ncol': 2}

zipcodes.plot(column = 'zip', figsize=(10, 10),
              edgecolor = 'black',
              legend = True, legend_kwds = leg_kwds, 
              cmap = 'Pastel1')
plt.show()

In [ ]:
bus_stops = pd.read_csv('../data/busstops_cleaned.csv') # from ea
print(bus_stops.shape)
bus_stops.head(3)

In [ ]:
bus_stops['geometry'] = bus_stops.apply(lambda x: Point((float(x.lng), 
                                                         float(x.lat))), 
                                        axis=1)
bus_stops.head(3)

In [ ]:
bus_geo = gpd.GeoDataFrame(bus_stops, 
                           crs = zipcodes.crs, 
                           geometry = bus_stops['geometry'])

In [ ]:
zipcodes = zipcodes[['zip', 'po_name', 'geometry']]
zipcodes

In [ ]:
stops_by_zip = gpd.sjoin(bus_geo, zipcodes, op = 'within')

In [ ]:
trip_starts_by_zip = gpd.sjoin(trips_cleaned_starts, zipcodes, op = 'within')

In [ ]:
trip_ends_by_zip = gpd.sjoin(trips_cleaned_ends, zipcodes, op = 'within')

In [ ]:
trip_starts_by_zip['zip'].value_counts()

In [ ]:
trip_ends_by_zip['zip'].value_counts()

In [ ]:
stops_in_37207 = stops_by_zip.loc[stops_by_zip['zip'] == '37207']
stops_in_37207.shape

In [ ]:
trip_starts_in_37207 = trip_starts_by_zip.loc[trip_starts_by_zip['zip'] == '37207']
trip_starts_in_37207.shape

In [ ]:
trip_ends_in_37207 = trip_ends_by_zip.loc[trip_ends_by_zip['zip'] == '37207']
trip_ends_in_37207.shape

In [ ]:
trip_starts_in_37207 = gpd.GeoDataFrame(trip_starts_in_37207, 
                           crs = zipcodes.crs, 
                           geometry = trip_starts_in_37207['geometry'])

In [ ]:
trip_ends_in_37207 = gpd.GeoDataFrame(trip_ends_in_37207, 
                           crs = zipcodes.crs, 
                           geometry = trip_ends_in_37207['geometry'])

In [ ]:
polygon37207 = zipcodes.loc[zipcodes['zip'] == '37207']
polygon37207.shape

In [ ]:
#fig, ax = matplotlib.subplots()
ax = polygon37207.plot(figsize = (8, 10), color = 'lightgreen')
stops_in_37207.plot( ax = ax, column = 'route');
trip_starts_in_37207.plot( ax = ax, column = 'triprecordnum');
plt.show();

In [ ]:
polygon37207.geometry.centroid

In [ ]:
#find the center of the larger (more southern) polygon and look at it
center = polygon37207.geometry.centroid[42]
print(center)

In [ ]:
#find the center of the larger (more southern) polygon and look at it
center = polygon37207.geometry.centroid[42]
print(center)

In [ ]:
# reverse the order when constructing the array for folium location
area_center = [center.y, center.x]

# check the order of area_center, the location for our folium map
print(area_center)

In [ ]:
#create our map of Nashville Zip Code 37207 and show it
map_37207 = folium.Map(location =  area_center, zoom_start = 12)
map_37207

In [ ]:
i = 0
for row_index, row_values in stops_in_37207.iterrows():
    if i <3: 
        print('index is', row_index)
        print('values are:')
        print(' ')
        print(row_values)
        print('------------------------- ')
        i+=1

In [ ]:
i = 0
for row_index, row_values in trip_starts_in_37207.iterrows():
    if i <3: 
        print('index is', row_index)
        print('values are:')
        print(' ')
        print(row_values)
        print('------------------------- ')
        i+=1

In [ ]:
cluster_map_37207 = folium.Map(location =  area_center, zoom_start = 12)

#create a marker cluster
marker_cluster = MarkerCluster().add_to(cluster_map_37207)

folium.GeoJson(polygon37207).add_to(cluster_map_37207)

# inside the loop add each marker to the cluster
for row_index, row_values in stops_in_37207.iterrows():
    loc = [row_values['lat'], row_values['lng']]
    pop = str(row_values['route'])
    icon=folium.Icon(color="blue",icon="bus", prefix='fa')
    
    marker = folium.Marker(
       location = loc, 
       popup = pop,
     icon = icon) 
    
    marker.add_to(marker_cluster)  
    
for row_index, row_values in trip_starts_in_37207.iterrows():
    loc = [row_values['startlatitude'], row_values['startlongitude']]
    pop = str(row_values['triprecordnum'])
    icon=folium.Icon(color="green",icon="scooter", prefix='fa')
    
    
    marker = folium.Marker(
        location = loc, 
        popup = pop,
    icon = icon) 
    
    marker.add_to(marker_cluster)

    
#save an interactive HTML map by calling .save()
cluster_map_37207.save('../maps/trip_starts_cluster37207.html')

cluster_map_37207  

In [ ]:
trip_ends_cluster_map_37207 = folium.Map(location =  area_center, zoom_start = 12)

#create a marker cluster
marker_cluster = MarkerCluster().add_to(trip_ends_cluster_map_37207)

folium.GeoJson(polygon37207).add_to(trip_ends_cluster_map_37207)

# inside the loop add each marker to the cluster
for row_index, row_values in stops_in_37207.iterrows():
    loc = [row_values['lat'], row_values['lng']]
    pop = str(row_values['route'])
    icon=folium.Icon(color="blue",icon="bus", prefix='fa')
    
    marker = folium.Marker(
       location = loc, 
       popup = pop,
     icon = icon) 
    
    marker.add_to(marker_cluster)  
    
for row_index, row_values in trip_ends_in_37207.iterrows():
    loc = [row_values['endlatitude'], row_values['endlongitude']]
    pop = str(row_values['triprecordnum'])
    icon=folium.Icon(color="red",icon="scooter", prefix='fa')
    
    
    marker = folium.Marker(
        location = loc, 
        popup = pop,
    icon = icon) 
    
    marker.add_to(marker_cluster)

    
#save an interactive HTML map by calling .save()
trip_ends_cluster_map_37207.save('../maps/trip_ends_cluster37207.html')

trip_ends_cluster_map_37207  

## ZIPCODE - 37201

In [ ]:
stops_in_37201 = stops_by_zip.loc[stops_by_zip['zip'] == '37201']
stops_in_37201.shape

In [ ]:
trip_starts_in_37201 = trip_starts_by_zip.loc[trip_starts_by_zip['zip'] == '37201']
trip_starts_in_37201.shape

In [ ]:
trip_ends_in_37201 = trip_ends_by_zip.loc[trip_ends_by_zip['zip'] == '37201']
trip_ends_in_37201.shape

In [ ]:
trip_starts_in_37201 = gpd.GeoDataFrame(trip_starts_in_37201, 
                           crs = zipcodes.crs, 
                           geometry = trip_starts_in_37201['geometry'])

In [ ]:
trip_ends_in_37201 = gpd.GeoDataFrame(trip_ends_in_37201, 
                           crs = zipcodes.crs, 
                           geometry = trip_ends_in_37201['geometry'])

In [ ]:
polygon37201 = zipcodes.loc[zipcodes['zip'] == '37201']
polygon37201.shape

In [ ]:
polygon37201.geometry.centroid

In [ ]:
center_37201 = polygon37201.geometry.centroid
print(center_37201)

In [ ]:
# reverse the order when constructing the array for folium location
area_center_37201 = [center_37201.y, center_37201.x]

# check the order of area_center, the location for our folium map
print(area_center_37201)

In [ ]:
#create our map of Nashville Zip Code 37201 and show it
map_37201 = folium.Map(location =  area_center, zoom_start = 12)
map_37201

In [ ]:
trip_starts_cluster_map_37201 = folium.Map(location =  area_center_37201, zoom_start = 12)

#create a marker cluster
marker_cluster = MarkerCluster().add_to(trip_starts_cluster_map_37201)

folium.GeoJson(polygon37201).add_to(trip_starts_cluster_map_37201)

# inside the loop add each marker to the cluster
for row_index, row_values in stops_in_37201.iterrows():
    loc = [row_values['lat'], row_values['lng']]
    pop = str(row_values['route'])
    icon=folium.Icon(color="blue",icon="bus", prefix='fa')
    
    marker = folium.Marker(
       location = loc, 
       popup = pop,
     icon = icon) 
    
    marker.add_to(marker_cluster)  
    
for row_index, row_values in trip_starts_in_37201.iterrows():
    loc = [row_values['startlatitude'], row_values['startlongitude']]
    pop = str(row_values['triprecordnum'])
    icon=folium.Icon(color="green",icon="scooter", prefix='fa')
    
    
    marker = folium.Marker(
        location = loc, 
        popup = pop,
    icon = icon) 
    
    marker.add_to(marker_cluster)

    
#save an interactive HTML map by calling .save()
trip_starts_cluster_map_37201.save('../maps/trip_starts_cluster37201.html')

trip_starts_cluster_map_37201  

In [ ]:
trip_ends_cluster_map_37201 = folium.Map(location =  area_center_37201, zoom_start = 12)

#create a marker cluster
marker_cluster = MarkerCluster().add_to(trip_ends_cluster_map_37201)

folium.GeoJson(polygon37201).add_to(trip_ends_cluster_map_37201)

# inside the loop add each marker to the cluster
for row_index, row_values in stops_in_37201.iterrows():
    loc = [row_values['lat'], row_values['lng']]
    pop = str(row_values['route'])
    icon=folium.Icon(color="blue",icon="bus", prefix='fa')
    
    marker = folium.Marker(
       location = loc, 
       popup = pop,
     icon = icon) 
    
    marker.add_to(marker_cluster)  
    
for row_index, row_values in trip_ends_in_37201.iterrows():
    loc = [row_values['endlatitude'], row_values['endlongitude']]
    pop = str(row_values['triprecordnum'])
    icon=folium.Icon(color="red",icon="scooter", prefix='fa')
    
    
    marker = folium.Marker(
        location = loc, 
        popup = pop,
    icon = icon) 
    
    marker.add_to(marker_cluster)

    
#save an interactive HTML map by calling .save()
trip_ends_cluster_map_37201.save('../maps/trip_ends_cluster37201.html')

trip_ends_cluster_map_37201  

# ZIPCODE - 37219

In [ ]:
stops_in_37219 = stops_by_zip.loc[stops_by_zip['zip'] == '37219']
stops_in_37219.shape

In [ ]:
trip_starts_in_37219 = trip_starts_by_zip.loc[trip_starts_by_zip['zip'] == '37219']
trip_starts_in_37219.shape

In [ ]:
trip_ends_in_37219 = trip_ends_by_zip.loc[trip_ends_by_zip['zip'] == '37219']
trip_ends_in_37219.shape

In [ ]:
trip_starts_in_37219 = gpd.GeoDataFrame(trip_starts_in_37219, 
                           crs = zipcodes.crs, 
                           geometry = trip_starts_in_37219['geometry'])

In [ ]:
trip_ends_in_37219 = gpd.GeoDataFrame(trip_ends_in_37219, 
                           crs = zipcodes.crs, 
                           geometry = trip_ends_in_37219['geometry'])

In [ ]:
polygon37219 = zipcodes.loc[zipcodes['zip'] == '37219']
polygon37219.shape

In [ ]:
polygon37219.geometry.centroid

In [ ]:
center_37219 = polygon37219.geometry.centroid
print(center_37219)

In [ ]:
# reverse the order when constructing the array for folium location
area_center_37219 = [center_37219.y, center_37219.x]

# check the order of area_center, the location for our folium map
print(area_center_37219)

In [ ]:
#create our map of Nashville Zip Code 37219 and show it
map_37219 = folium.Map(location =  area_center_37219, zoom_start = 12)
map_37219

In [ ]:
trip_starts_cluster_map_37219 = folium.Map(location =  area_center_37219, zoom_start = 12)

#create a marker cluster
marker_cluster = MarkerCluster().add_to(trip_starts_cluster_map_37219)

folium.GeoJson(polygon37219).add_to(trip_starts_cluster_map_37219)

# inside the loop add each marker to the cluster
for row_index, row_values in stops_in_37219.iterrows():
    loc = [row_values['lat'], row_values['lng']]
    pop = str(row_values['route'])
    icon=folium.Icon(color="blue",icon="bus", prefix='fa')
    
    marker = folium.Marker(
       location = loc, 
       popup = pop,
     icon = icon) 
    
    marker.add_to(marker_cluster)  
    
for row_index, row_values in trip_starts_in_37219.iterrows():
    loc = [row_values['startlatitude'], row_values['startlongitude']]
    pop = str(row_values['triprecordnum'])
    icon=folium.Icon(color="green",icon="scooter", prefix='fa')
    
    
    marker = folium.Marker(
        location = loc, 
        popup = pop,
    icon = icon) 
    
    marker.add_to(marker_cluster)

    
#save an interactive HTML map by calling .save()
trip_starts_cluster_map_37219.save('../maps/trip_starts_cluster37219.html')

trip_starts_cluster_map_37219 

In [ ]:
trip_ends_cluster_map_37219 = folium.Map(location =  area_center_37219, zoom_start = 12)

#create a marker cluster
marker_cluster = MarkerCluster().add_to(trip_ends_cluster_map_37219)

folium.GeoJson(polygon37219).add_to(trip_ends_cluster_map_37219)

# inside the loop add each marker to the cluster
for row_index, row_values in stops_in_37219.iterrows():
    loc = [row_values['lat'], row_values['lng']]
    pop = str(row_values['route'])
    icon=folium.Icon(color="blue",icon="bus", prefix='fa')
    
    marker = folium.Marker(
       location = loc, 
       popup = pop,
     icon = icon) 
    
    marker.add_to(marker_cluster)  
    
for row_index, row_values in trip_ends_in_37219.iterrows():
    loc = [row_values['endlatitude'], row_values['endlongitude']]
    pop = str(row_values['triprecordnum'])
    icon=folium.Icon(color="red",icon="scooter", prefix='fa')
    
    
    marker = folium.Marker(
        location = loc, 
        popup = pop,
    icon = icon) 
    
    marker.add_to(marker_cluster)

    
#save an interactive HTML map by calling .save()
trip_ends_cluster_map_37219.save('../maps/trip_ends_cluster37219.html')

trip_ends_cluster_map_37219  

# ZIPCODE - 37212

In [ ]:
stops_in_37212 = stops_by_zip.loc[stops_by_zip['zip'] == '37212']
stops_in_37212.shape

In [ ]:
trip_starts_in_37212 = trip_starts_by_zip.loc[trip_starts_by_zip['zip'] == '37212']
trip_starts_in_37212.shape

In [ ]:
trip_ends_in_37212 = trip_ends_by_zip.loc[trip_ends_by_zip['zip'] == '37212']
trip_ends_in_37212.shape

In [ ]:
trip_starts_in_37212 = gpd.GeoDataFrame(trip_starts_in_37212, 
                           crs = zipcodes.crs, 
                           geometry = trip_starts_in_37212['geometry'])

In [ ]:
trip_ends_in_37212 = gpd.GeoDataFrame(trip_ends_in_37212, 
                           crs = zipcodes.crs, 
                           geometry = trip_ends_in_37212['geometry'])

In [ ]:
polygon37212 = zipcodes.loc[zipcodes['zip'] == '37212']
polygon37212.shape

In [ ]:
polygon37212.geometry.centroid

In [ ]:
center_37212 = polygon37212.geometry.centroid
print(center_37212)

In [ ]:
# reverse the order when constructing the array for folium location
area_center_37212 = [center_37212.y, center_37212.x]

# check the order of area_center, the location for our folium map
print(area_center_37212)

In [ ]:
#create our map of Nashville Zip Code 37219 and show it
map_37212 = folium.Map(location =  area_center_37212, zoom_start = 12)
map_37212

In [ ]:
trip_starts_cluster_map_37212 = folium.Map(location =  area_center_37212, zoom_start = 12)

#create a marker cluster
marker_cluster = MarkerCluster().add_to(trip_starts_cluster_map_37212)

folium.GeoJson(polygon37212).add_to(trip_starts_cluster_map_37212)

# inside the loop add each marker to the cluster
for row_index, row_values in stops_in_37212.iterrows():
    loc = [row_values['lat'], row_values['lng']]
    pop = str(row_values['route'])
    icon=folium.Icon(color="blue",icon="bus", prefix='fa')
    
    marker = folium.Marker(
       location = loc, 
       popup = pop,
     icon = icon) 
    
    marker.add_to(marker_cluster)  
    
for row_index, row_values in trip_starts_in_37212.iterrows():
    loc = [row_values['startlatitude'], row_values['startlongitude']]
    pop = str(row_values['triprecordnum'])
    icon=folium.Icon(color="green",icon="scooter", prefix='fa')
    
    
    marker = folium.Marker(
        location = loc, 
        popup = pop,
    icon = icon) 
    
    marker.add_to(marker_cluster)

    
#save an interactive HTML map by calling .save()
trip_starts_cluster_map_37212.save('../maps/trip_starts_cluster37212.html')

trip_starts_cluster_map_37212 

In [ ]:
trip_ends_cluster_map_37212 = folium.Map(location =  area_center_37212, zoom_start = 12)

#create a marker cluster
marker_cluster = MarkerCluster().add_to(trip_ends_cluster_map_37212)

folium.GeoJson(polygon37212).add_to(trip_ends_cluster_map_37212)

# inside the loop add each marker to the cluster
for row_index, row_values in stops_in_37212.iterrows():
    loc = [row_values['lat'], row_values['lng']]
    pop = str(row_values['route'])
    icon=folium.Icon(color="blue",icon="bus", prefix='fa')
    
    marker = folium.Marker(
       location = loc, 
       popup = pop,
     icon = icon) 
    
    marker.add_to(marker_cluster)  
    
for row_index, row_values in trip_ends_in_37212.iterrows():
    loc = [row_values['endlatitude'], row_values['endlongitude']]
    pop = str(row_values['triprecordnum'])
    icon=folium.Icon(color="red",icon="scooter", prefix='fa')
    
    
    marker = folium.Marker(
        location = loc, 
        popup = pop,
    icon = icon) 
    
    marker.add_to(marker_cluster)

    
#save an interactive HTML map by calling .save()
trip_ends_cluster_map_37212.save('../maps/trip_ends_cluster37212.html')

trip_ends_cluster_map_37212  

# ZIPCODE - 37208

In [ ]:
stops_in_37208 = stops_by_zip.loc[stops_by_zip['zip'] == '37208']
stops_in_37208.shape

In [ ]:
trip_starts_in_37208 = trip_starts_by_zip.loc[trip_starts_by_zip['zip'] == '37208']
trip_starts_in_37208.shape

In [ ]:
trip_ends_in_37208 = trip_ends_by_zip.loc[trip_ends_by_zip['zip'] == '37208']
trip_ends_in_37208.shape

In [ ]:
trip_starts_in_37208 = gpd.GeoDataFrame(trip_starts_in_37208, 
                           crs = zipcodes.crs, 
                           geometry = trip_starts_in_37208['geometry'])

In [ ]:
trip_ends_in_37208 = gpd.GeoDataFrame(trip_ends_in_37208, 
                           crs = zipcodes.crs, 
                           geometry = trip_ends_in_37208['geometry'])

In [ ]:
polygon37208 = zipcodes.loc[zipcodes['zip'] == '37208']
polygon37208.shape

In [ ]:
polygon37208.geometry.centroid

In [ ]:
center_37208 = polygon37208.geometry.centroid
print(center_37208)

In [ ]:
# reverse the order when constructing the array for folium location
area_center_37208 = [center_37208.y, center_37208.x]

# check the order of area_center, the location for our folium map
print(area_center_37208)

In [ ]:
#create our map of Nashville Zip Code 37219 and show it
map_37208 = folium.Map(location =  area_center_37208, zoom_start = 12)
map_37208

In [ ]:
trip_starts_cluster_map_37208 = folium.Map(location =  area_center_37208, zoom_start = 12)

#create a marker cluster
marker_cluster = MarkerCluster().add_to(trip_starts_cluster_map_37208)

folium.GeoJson(polygon37208).add_to(trip_starts_cluster_map_37208)

# inside the loop add each marker to the cluster
for row_index, row_values in stops_in_37208.iterrows():
    loc = [row_values['lat'], row_values['lng']]
    pop = str(row_values['route'])
    icon=folium.Icon(color="blue",icon="bus", prefix='fa')
    
    marker = folium.Marker(
       location = loc, 
       popup = pop,
     icon = icon) 
    
    marker.add_to(marker_cluster)  
    
for row_index, row_values in trip_starts_in_37208.iterrows():
    loc = [row_values['startlatitude'], row_values['startlongitude']]
    pop = str(row_values['triprecordnum'])
    icon=folium.Icon(color="green",icon="scooter", prefix='fa')
    
    
    marker = folium.Marker(
        location = loc, 
        popup = pop,
    icon = icon) 
    
    marker.add_to(marker_cluster)

    
#save an interactive HTML map by calling .save()
trip_starts_cluster_map_37208.save('../maps/trip_starts_cluster37208.html')

trip_starts_cluster_map_37208

In [ ]:
trip_ends_cluster_map_37208 = folium.Map(location =  area_center_37208, zoom_start = 12)

#create a marker cluster
marker_cluster = MarkerCluster().add_to(trip_ends_cluster_map_37208)

folium.GeoJson(polygon37208).add_to(trip_ends_cluster_map_37208)

# inside the loop add each marker to the cluster
for row_index, row_values in stops_in_37208.iterrows():
    loc = [row_values['lat'], row_values['lng']]
    pop = str(row_values['route'])
    icon=folium.Icon(color="blue",icon="bus", prefix='fa')
    
    marker = folium.Marker(
       location = loc, 
       popup = pop,
     icon = icon) 
    
    marker.add_to(marker_cluster)  
    
for row_index, row_values in trip_ends_in_37208.iterrows():
    loc = [row_values['endlatitude'], row_values['endlongitude']]
    pop = str(row_values['triprecordnum'])
    icon=folium.Icon(color="red",icon="scooter", prefix='fa')
    
    
    marker = folium.Marker(
        location = loc, 
        popup = pop,
    icon = icon) 
    
    marker.add_to(marker_cluster)

    
#save an interactive HTML map by calling .save()
trip_ends_cluster_map_37208.save('../maps/trip_ends_cluster37208.html')

trip_ends_cluster_map_37208

# ZIPCODE = 37203

In [ ]:
stops_in_37203 = stops_by_zip.loc[stops_by_zip['zip'] == '37203']
stops_in_37203.shape

In [ ]:
trip_starts_in_37203 = trip_starts_by_zip.loc[trip_starts_by_zip['zip'] == '37203']
trip_starts_in_37203.shape

In [ ]:
trip_ends_in_37203 = trip_ends_by_zip.loc[trip_ends_by_zip['zip'] == '37203']
trip_ends_in_37203.shape

In [ ]:
trip_starts_in_37203 = gpd.GeoDataFrame(trip_starts_in_37203, 
                           crs = zipcodes.crs, 
                           geometry = trip_starts_in_37203['geometry'])

In [ ]:
trip_ends_in_37203 = gpd.GeoDataFrame(trip_ends_in_37203, 
                           crs = zipcodes.crs, 
                           geometry = trip_ends_in_37203['geometry'])

In [ ]:
polygon37203 = zipcodes.loc[zipcodes['zip'] == '37203']
polygon37203.shape

In [ ]:
polygon37203.geometry.centroid

In [ ]:
center_37203 = polygon37203.geometry.centroid
print(center_37208)

In [ ]:
# reverse the order when constructing the array for folium location
area_center_37203 = [center_37203.y, center_37203.x]

# check the order of area_center, the location for our folium map
print(area_center_37203)

In [ ]:
#create our map of Nashville Zip Code 37219 and show it
map_37203 = folium.Map(location =  area_center_37203, zoom_start = 12)
map_37203

In [ ]:
trip_starts_cluster_map_37203 = folium.Map(location =  area_center_37203, zoom_start = 12)

#create a marker cluster
marker_cluster = MarkerCluster().add_to(trip_starts_cluster_map_37203)

folium.GeoJson(polygon37203).add_to(trip_starts_cluster_map_37203)

# inside the loop add each marker to the cluster
for row_index, row_values in stops_in_37203.iterrows():
    loc = [row_values['lat'], row_values['lng']]
    pop = str(row_values['route'])
    icon=folium.Icon(color="blue",icon="bus", prefix='fa')
    
    marker = folium.Marker(
       location = loc, 
       popup = pop,
     icon = icon) 
    
    marker.add_to(marker_cluster)  
    
for row_index, row_values in trip_starts_in_37203.iterrows():
    loc = [row_values['startlatitude'], row_values['startlongitude']]
    pop = str(row_values['triprecordnum'])
    icon=folium.Icon(color="green",icon="scooter", prefix='fa')
    
    
    marker = folium.Marker(
        location = loc, 
        popup = pop,
    icon = icon) 
    
    marker.add_to(marker_cluster)

    
#save an interactive HTML map by calling .save()
trip_starts_cluster_map_37203.save('../maps/trip_starts_cluster37203.html')

trip_starts_cluster_map_37203

In [ ]:
trip_ends_cluster_map_37203 = folium.Map(location =  area_center_37203, zoom_start = 12)

#create a marker cluster
marker_cluster = MarkerCluster().add_to(trip_ends_cluster_map_37203)

folium.GeoJson(polygon37203).add_to(trip_ends_cluster_map_37203)

# inside the loop add each marker to the cluster
for row_index, row_values in stops_in_37203.iterrows():
    loc = [row_values['lat'], row_values['lng']]
    pop = str(row_values['route'])
    icon=folium.Icon(color="blue",icon="bus", prefix='fa')
    
    marker = folium.Marker(
       location = loc, 
       popup = pop,
     icon = icon) 
    
    marker.add_to(marker_cluster)  
    
for row_index, row_values in trip_ends_in_37203.iterrows():
    loc = [row_values['endlatitude'], row_values['endlongitude']]
    pop = str(row_values['triprecordnum'])
    icon=folium.Icon(color="red",icon="scooter", prefix='fa')
    
    
    marker = folium.Marker(
        location = loc, 
        popup = pop,
    icon = icon) 
    
    marker.add_to(marker_cluster)

    
#save an interactive HTML map by calling .save()
trip_ends_cluster_map_37203.save('../maps/trip_ends_cluster37203.html')

trip_ends_cluster_map_37203

3. The goal of Metro Nashville is to have each scooter used a minimum of 3 times per day. Based on the data, what is the average number of trips per scooter per day? Make sure to consider the days that a scooter was available. How does this vary by company?

4. What is the highest count of scooters being used at the same time? When did it occur? Does this vary by zip code or other geographic region?

5. SUMDs can provide alternative transportation and provide "last mile" access to public transit. How often are trips starting near public transit hubs? You can download a dataset of bus stop locations from https://data.nashville.gov/Transportation/Regional-Transportation-Authority-Bus-Stops/p886-fnbd.

Deliverables:
At the conclusion of this project, your group should deliver a presentation which addresses the following points:
* Are scooter companies in compliance with the required data cleaning?
* What are typical usage patterns for scooters in terms of time, location, and trip duration?
* Does it appear that scooters are used as "last mile" transportation from public transit hubs to work or school?
* What are your recommendations for total number of scooters for the city overall and density of scooters by zip code?